In [ ]:
def variable_summaries(var, name):
    '''Attach a lot of summaries to a Tensor.'''
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)

In [18]:
# Parameters
batch_size = 3
seq_length = 6
rnn_size   = 10
vocab_size = 5
save_dir_GAN = 'models_GAN'
vocab_file = 'simple_vocab.pkl'

In [5]:
# Load Vocabulary
def load_vocab(save_dir_GAN, vocab_file):
    with open(os.path.join(save_dir_GAN, vocab_file)) as f:
            chars, vocab = cPickle.load(f)
    return chars, vocab

In [47]:
# Generator
import tensorflow as tf
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()

def map_to_text(indices, chars):
    text = []
    for index in indices:
        text.append(chars[index])
    return text

def generate():    
    # Vocabulary
#     chars, vocab = load_vocab(save_dir_GAN, vocab_file)

    # Initial indices
    indices = tf.constant(3, shape=[batch_size])

    # Embeddings and Logits
    embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
    softmax   = tf.get_variable('softmax', [rnn_size, vocab_size])

    # RNN
    outputs, text = [], []
    cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
    state = cell.zero_state(batch_size, tf.float32)
    inp = tf.nn.embedding_lookup(embedding, indices)
        
    with tf.variable_scope('rnn'):
        for i in xrange(seq_length):
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            rnn_out, state = cell(inp, state)
            logits_out = tf.matmul(rnn_out, softmax)
            outputs.append(logits_out)
            indices = tf.squeeze(Categorical(logits_out).sample(n=1))
            inp = tf.nn.embedding_lookup(embedding, indices)
    return outputs
            

with tf.Session() as sess: 
    outputs = generate()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'

Iteration 0
[[-0.34617978  0.20336334  0.02268175 -0.04673538 -0.28929198]
 [-0.34617978  0.20336334  0.02268175 -0.04673538 -0.28929198]
 [-0.34617978  0.20336334  0.02268175 -0.04673538 -0.28929198]] 

Iteration 1
[[ 0.35516712  0.10131244  0.07381877 -0.35479796  0.16101244]
 [ 0.05509398  0.08782202  0.21242316 -0.45634913 -0.45755401]
 [-0.1003852   0.17804173  0.27465442 -0.46560803 -0.6932022 ]] 

Iteration 2
[[ 0.19129771 -0.22701152  0.02287175 -0.70750898 -0.77059513]
 [ 0.36946377 -0.18053031  0.00662056  0.02199733  0.35659   ]
 [ 0.317045   -0.25418457 -0.01815362 -0.18503408  0.11885534]] 

Iteration 3
[[ 0.22103436 -0.0832421   0.05149586  0.24169527  0.43850553]
 [ 0.38890791 -0.14602259 -0.02570416 -0.51335734 -0.66373748]
 [ 0.36453071 -0.27231413 -0.13302399 -0.61939329 -0.72989237]] 

Iteration 4
[[ 0.17357416  0.15284172  0.10503966 -0.33290198  0.13347571]
 [ 0.01823761 -0.15699735 -0.25087681 -0.32819456 -0.53536671]
 [ 0.16192679 -0.19149378 -0.03260802  0.12183

In [29]:
tf.reset_default_graph()
logits = tf.constant([1.,2.,3.])
indices = tf.squeeze(Categorical(logits).sample(n=5))

with tf.Session() as sess:
    print 'Indices: ',indices.eval()

Indices:  [2 2 2 2 1]


In [41]:
import os
import cPickle

chars, vocab = load_vocab('models_GAN','simple_vocab.pkl')
chars
# vocab

('e', ' ', 'r', 'b', '\n')

In [ ]:
# Train Generator
def train_generator():
    pass

In [ ]:
# Train Discriminator
def train_discriminator():
    pass